<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Procesamiento de Lenguaje Natural</h1>

<h3>LSTM básico: Haciendo N-gramas</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/deep-ngram.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

Tomado parcialmente y adaptado de [el repositorio de github](https://github.com/shaundsouza/lstm-textual-ngrams) del trabajo [*LSTM Neural Network for Textual Ngrams* (D'Souza, 2018)](https://www.preprints.org/manuscript/201811.0579/v1)



</center>


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

## Importando datos de alguna obra 

Vamos a descargar un *Corpus*, y pues vamos a usar algo muy famoso, como  puede ser la obra de Shakespiare en inglés, o *El Quijote* en español. Empecemos por *El Quijote*

In [ ]:
!curl -o quijote.txt https://www.gutenberg.org/cache/epub/2000/pg2000.txt

y ahora si vamos a ver como nos quedaría el texto.

In [ ]:
archivo = "quijote.txt"
with open(archivo, 'r', encoding='utf8') as fp:
    corpus = fp.read()

print(corpus)

## Preprocesando la información para ser utilizada en un modelo tipo LSTM



Vamos a generar dos íncides, de indices a caracteres y de caracteres a ´ndices. Y sin dar ni media pasada de limpieza. Ya veremos si limpiamos el texto como queda:

In [ ]:
chars = sorted(list(set(corpus)))
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

print('# de caracteres:', len(chars))
print(char_indices.keys())
for i in range(20):
  print(f"indices_char[{i}] = {indices_char[i]}")


Y ahora vamos a dividir entre entradas y salidas, donde cada entrada es una cadena de maxlen caracteres, la salida es el próximo caracter, y vamos avanzando en el texto en saltos (`step`).

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i: i + maxlen])
    next_chars.append(corpus[i + maxlen])

print(sentences[:10])
print(next_chars[:10])
print('nb sequences:', len(sentences))

Y ahora vamos a convertirlos a tensores, usando la codificación de `one hot encoding`, que es la más malita posible:

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# build the model: a single LSTM
print("x ", x.shape)
print("y ", y.shape)

## Modelado neuronal

Definamos un modelo bastante simple:

In [ ]:
modelo = keras.Sequential([
  layers.LSTM(128, input_shape=(maxlen, len(chars))),
  layers.Dense(len(chars), activation='softmax')
])

y lo entrenamos

In [ ]:
modelo.compile(
    loss='categorical_crossentropy', 
    optimizer='adam',
    metrics=['accuracy']
)

## Predicciones

En este caso, lo que queremos es que el sistema realice una predicción simple, usando los valores pasados para predecir el nuevo:

In [ ]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_text(modelo, inicial, caracteres=10, char_indices, indices_char temperature=1.0):
  generado = inicial
  for i in range(caracteres):
    x_pred = np.zeros((1, maxlen, len(chars)))

    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1.

    preds = modelo.predict(x_pred)[0]
    next_index = sample(preds, temperature)
    next_char = indices_char[next_index]

    generado += [next_char]
  return generado
          


Vamos a ver un ejemplo:

In [ ]:
T = 1
inicial = "De verdad os digo que"

a = predict_text(
    modelo, 
    inicial, 
    caracteres=20, 
    char_indices, 
    indices_char,
    temperature=T)

print(a)